In [75]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import statsmodels.api as sm
import seaborn as sns
import re
from scipy.stats import shapiro
sns.set()
from pylab import rcParams
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
rcParams['figure.figsize'] = 15, 10

In [76]:
df = pd.read_csv('nyc_rolling_sales_cleaned2.csv')
# df.reset_index(drop = True, inplace = True)

In [77]:
df = df.drop([df.columns[0]], axis=1)

In [78]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,Manhattan,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,C2,153 AVENUE B,10009,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19
1,Manhattan,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,C4,154 EAST 7TH STREET,10009,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23
2,Manhattan,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,C2,301 EAST 10TH STREET,10009,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17
3,Manhattan,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,406,32,C4,210 AVENUE B,10009,8,0,8,1750,4226,1920,2,C4,3192840,2016-09-23
4,Manhattan,ALPHABET CITY,08 RENTALS - ELEVATOR APARTMENTS,2,387,153,D9,629 EAST 5TH STREET,10009,24,0,24,4489,18523,1920,2,D9,16232000,2016-11-07


In [79]:
print("Sale Prices Range from: ", df['SALE PRICE'].min(), " to ", df['SALE PRICE'].max())
print("LAND SQUARE FEET Range from: ", df['LAND SQUARE FEET'].min(), " to ", df['LAND SQUARE FEET'].max())
print("GROSS SQUARE FEET Range from: ", df['GROSS SQUARE FEET'].min(), " to ", df['GROSS SQUARE FEET'].max())
print("YEAR BUILT Range from: ", df['YEAR BUILT'].min(), " to ", df['YEAR BUILT'].max())

Sale Prices Range from:  1000  to  2210000000
LAND SQUARE FEET Range from:  33  to  4252327
GROSS SQUARE FEET Range from:  0  to  3750565
YEAR BUILT Range from:  1800  to  2017


In [80]:
df["Sale Price Class"] = df["SALE PRICE"]/1000000
df["Sale Price Class"] = df["Sale Price Class"].astype('int')

df["LAND SQUARE FEET"] = df["LAND SQUARE FEET"]/1000
df["LAND SQUARE FEET"] = df["LAND SQUARE FEET"].astype('int')

df["GROSS SQUARE FEET"] = df["GROSS SQUARE FEET"]/1000
df["GROSS SQUARE FEET"] = df["GROSS SQUARE FEET"].astype('int')

df["YEAR BUILT"] = (df["YEAR BUILT"] - 1800)/50
df["YEAR BUILT"] = df["YEAR BUILT"].astype('int')

df.head(5)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,Sale Price Class
0,Manhattan,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,C2,153 AVENUE B,10009,5,0,5,1,6,2,2,C2,6625000,2017-07-19,6
1,Manhattan,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,C4,154 EAST 7TH STREET,10009,10,0,10,2,6,2,2,C4,3936272,2016-09-23,3
2,Manhattan,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,C2,301 EAST 10TH STREET,10009,6,0,6,2,4,2,2,C2,8000000,2016-11-17,8
3,Manhattan,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,406,32,C4,210 AVENUE B,10009,8,0,8,1,4,2,2,C4,3192840,2016-09-23,3
4,Manhattan,ALPHABET CITY,08 RENTALS - ELEVATOR APARTMENTS,2,387,153,D9,629 EAST 5TH STREET,10009,24,0,24,4,18,2,2,D9,16232000,2016-11-07,16


In [81]:
df.dtypes

BOROUGH                           object
NEIGHBORHOOD                      object
BUILDING CLASS CATEGORY           object
TAX CLASS AT PRESENT              object
BLOCK                              int64
LOT                                int64
BUILDING CLASS AT PRESENT         object
ADDRESS                           object
ZIP CODE                           int64
RESIDENTIAL UNITS                  int64
COMMERCIAL UNITS                   int64
TOTAL UNITS                        int64
LAND SQUARE FEET                   int32
GROSS SQUARE FEET                  int32
YEAR BUILT                         int32
TAX CLASS AT TIME OF SALE          int64
BUILDING CLASS AT TIME OF SALE    object
SALE PRICE                         int64
SALE DATE                         object
Sale Price Class                   int32
dtype: object

In [82]:
print("Sale Prices Classes Range from: ", df['Sale Price Class'].min(), " to ", df['Sale Price Class'].max())
print("LAND SQUARE FEET Range from: ", df['LAND SQUARE FEET'].min(), " to ", df['LAND SQUARE FEET'].max())
print("GROSS SQUARE FEET Range from: ", df['GROSS SQUARE FEET'].min(), " to ", df['GROSS SQUARE FEET'].max())
print("YEAR BUILT Range from: ", df['YEAR BUILT'].min(), " to ", df['YEAR BUILT'].max())

Sale Prices Classes Range from:  0  to  2210
LAND SQUARE FEET Range from:  0  to  4252
GROSS SQUARE FEET Range from:  0  to  3750
YEAR BUILT Range from:  0  to  4


In [83]:
for col in df.columns:
    if(col=="BOROUGH" or col=="NEIGHBORHOOD" or col=="BUILDING CLASS CATEGORY" or col=="TAX CLASS AT PRESENT" or col=="BUILDING CLASS AT PRESENT" or col=="TAX CLASS AT TIME OF SALE" or col=="BUILDING CLASS AT TIME OF SALE"):
        temp =  df[col].unique()
        for i in range(len(temp)):
            df[col] = df[col].replace(temp[i],i)

In [84]:
df['SALE YEAR'] = pd.DatetimeIndex(df['SALE DATE']).year

In [85]:
df['SALE YEAR']

0        2017
1        2016
2        2016
3        2016
4        2016
5        2016
6        2017
7        2017
8        2017
9        2017
10       2016
11       2017
12       2017
13       2017
14       2017
15       2017
16       2017
17       2017
18       2017
19       2017
20       2017
21       2017
22       2017
23       2016
24       2017
25       2017
26       2017
27       2017
28       2017
29       2017
         ... 
58651    2017
58652    2016
58653    2017
58654    2017
58655    2017
58656    2016
58657    2016
58658    2017
58659    2016
58660    2017
58661    2017
58662    2017
58663    2017
58664    2017
58665    2016
58666    2017
58667    2017
58668    2016
58669    2017
58670    2016
58671    2017
58672    2017
58673    2016
58674    2016
58675    2016
58676    2016
58677    2017
58678    2017
58679    2016
58680    2016
Name: SALE YEAR, Length: 58681, dtype: int64

In [86]:
for col in df.columns:
    if(col=="BOROUGH" or col=="NEIGHBORHOOD" or col=="BUILDING CLASS CATEGORY" or col=="TAX CLASS AT PRESENT" or col=="BUILDING CLASS AT PRESENT" or col=="TAX CLASS AT TIME OF SALE" or col=="BUILDING CLASS AT TIME OF SALE"):
        print(col) 
        print(df[col].unique())

BOROUGH
[0 1 2 3 4]
NEIGHBORHOOD
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 2

In [87]:
# df.reset_index(drop=True, inplace=True)
df.to_csv('nyc_rolling_sales_new1.csv')
df.dtypes

BOROUGH                            int64
NEIGHBORHOOD                       int64
BUILDING CLASS CATEGORY            int64
TAX CLASS AT PRESENT               int64
BLOCK                              int64
LOT                                int64
BUILDING CLASS AT PRESENT          int64
ADDRESS                           object
ZIP CODE                           int64
RESIDENTIAL UNITS                  int64
COMMERCIAL UNITS                   int64
TOTAL UNITS                        int64
LAND SQUARE FEET                   int32
GROSS SQUARE FEET                  int32
YEAR BUILT                         int32
TAX CLASS AT TIME OF SALE          int64
BUILDING CLASS AT TIME OF SALE     int64
SALE PRICE                         int64
SALE DATE                         object
Sale Price Class                   int32
SALE YEAR                          int64
dtype: object